## 🌐 Connect Colab to Google Drive

In [ ]:
from google.colab import drive

drive.mount("/gdrive")
%cd /gdrive/My Drive
%cd [2024-2025] AN2DL Homework 2

## ⚙️ Import Libraries

In [ ]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import logging
import random

import tensorflow as tf
from tensorflow import keras as tfk
tfk.config.enable_unsafe_deserialization()
from tensorflow.keras import layers as tfkl
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from scipy.stats import mode

import matplotlib.pyplot as plt
%matplotlib inline

seed = 29
np.random.seed(seed)
tf.random.set_seed(seed)

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

## ⏳ Load the Datasets

In [ ]:
data = np.load("mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Test X shape: {X_test.shape}")

# Add color channel and rescale pixels between 0 and 1
X_train = X_train[..., np.newaxis] / 255.0
X_test = X_test[..., np.newaxis] / 255.0

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")

## 🔍 Inspect the training dataset

In [ ]:
# Calculate prevalent labels
y_train_labels = mode(y_train, axis=(1, 2))[0].flatten()

print(f"Shape X_train: {X_train.shape}")
print(f"Shape y_train_labels: {y_train_labels.shape}")

# List all unique labels to check correctness
unique_labels = np.unique(y_train)
print(f"Unique classes: {unique_labels}")

# Plot images in batches
def plot_images(X, y, start_index=0, images_per_row=10, images_per_col=10):
    fig, axes = plt.subplots(images_per_col, images_per_row, figsize=(15, 15))
    for i in range(images_per_row * images_per_col):
        idx = start_index + i
        if idx >= len(X):
            break
        ax = axes[i // images_per_row, i % images_per_row]
        ax.imshow(X[idx], cmap="gray")
        ax.set_title(f"Class: {y[idx]}")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

# Plot a sample image from each class
def plot_one_sample_per_class(X, y, y_mask, classes):
    for label in classes:
        for i in range(len(y_mask)):
            if label in np.unique(y_mask[i]):
                plt.figure()
                plt.imshow(X[i], cmap="gray")
                plt.title(f"Class: {label}")
                plt.axis("off")
                plt.show()
                break

plot_one_sample_per_class(X_train, y_train_labels, y_train, unique_labels)

# Plot all images
images_per_row = 10
images_per_col = 10
images_per_page = images_per_row * images_per_col
num_images = X_train.shape[0]

for start_idx in range(0, num_images, images_per_page):
    plot_images(X_train, y_train_labels, start_index=start_idx, images_per_row=images_per_row, images_per_col=images_per_col)

## ❌ Remove outliers from dataset

In [ ]:
# Lists to contain filtered elements
X_train_filtered = []
y_train_filtered = []

for i in range(len(y_train)):
    label = y_train[i].argmax() if y_train.ndim > 1 else y_train[i]
    if label != 415:
        # Add to filtered dataset the non-alien images
        X_train_filtered.append(X_train[i])
        y_train_filtered.append(y_train[i])

# Convert lists to numpy arrays
X_train_filtered = np.array(X_train_filtered)
y_train_filtered = np.array(y_train_filtered)

print(f"Shape X_train_filtered: {X_train_filtered.shape}")
print(f"Shape y_train_filtered: {y_train_filtered.shape}")
print(f"Unique classes: {np.unique(y_train_filtered)}")

## 🔍 Inspect the filtered training dataset

In [ ]:
num_images_filtered = X_train_filtered.shape[0]
y_train_filtered_labels = mode(y_train_filtered, axis=(1, 2))[0].flatten()

# Plot the filtered dataset
for start_idx in range(0, num_images_filtered, images_per_page):
    plot_images(X_train_filtered, y_train_filtered_labels, start_index=start_idx, images_per_row=images_per_row, images_per_col=images_per_col)

## 🧮 Define network parameters

In [5]:
# Set batch size for training
batch_size = 64

# Set learning rate for the optimiser
learning_rate = 1e-4

# Set early stopping patience threshold
patience = 10

# Set maximum number of training epochs
epochs = 100

## ✂ Split into Training and Validation Sets

In [6]:
# Split the training dataset to get a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train_filtered,
    y_train_filtered,
    test_size=0.2,
    random_state=seed)

# Print the shapes of the resulting sets
print('Training set shape:\t', X_train.shape, y_train.shape)
print('Validation set shape:\t', X_val.shape, y_val.shape)

Training set shape:	 (2004, 64, 128, 1) (2004, 64, 128)
Validation set shape:	 (501, 64, 128, 1) (501, 64, 128)


## 🔄 Preprocess Dataset

In [7]:
def augment_data(image, label):
    # Geometric Transformations
    image = tf.image.random_flip_left_right(image)
    label = tf.image.random_flip_left_right(label)

    image = tf.image.random_flip_up_down(image)
    label = tf.image.random_flip_up_down(label)

    # Chromatic Transformations
    image = tf.image.random_brightness(image, max_delta=0.4)
    image = tf.image.random_contrast(image, lower=0.7, upper=1.3)

    return image, label

def preprocess_image(image):
    image = tf.expand_dims(image, axis=-1) if len(image.shape) == 2 else image
    image = tf.cast(image, tf.float32)
    return image

def preprocess_label(label):
    label = tf.expand_dims(label, axis=-1) if len(label.shape) == 2 else label
    label = tf.cast(label, tf.int32)
    return label

def preprocess_data(image, label):
    image = preprocess_image(image)
    label = preprocess_label(label)
    return image, label

In [8]:
# Dataset preprocessato (senza augmentation)
train_dataset_static = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset_static = train_dataset_static.map(preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset_static = train_dataset_static.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Dataset con augmentation
train_dataset_dynamic = train_dataset_static.map(lambda x, y: augment_data(x, y), num_parallel_calls=tf.data.AUTOTUNE)

# Dataset combinato
combined_dataset = train_dataset_static.concatenate(train_dataset_dynamic)
combined_dataset = combined_dataset.shuffle(buffer_size=len(X_train))
combined_dataset = combined_dataset.prefetch(tf.data.AUTOTUNE)

# Validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

## 🔍 Plot Datasets

In [ ]:
def plot_from_dataset(dataset, title, num_images=10):
    # Imposta il layout orizzontale
    fig, axes = plt.subplots(1, num_images, figsize=(num_images * 2, 3))
    fig.suptitle(title, fontsize=16, y=1.05)
    count = 0

    for batch in dataset:
        images, label_maps = batch  # Estrai immagini e feature map
        for i in range(len(images)):
            if count >= num_images:
                break

            # Prepara l'immagine
            image = images[i].numpy()
            if image.shape[-1] == 1:  # Scala di grigi
                image = tf.squeeze(image, axis=-1).numpy()

            # Mostra l'immagine
            axes[count].imshow(image, cmap='gray' if image.ndim == 2 else None, aspect='auto')
            axes[count].axis('off')

            count += 1

        if count >= num_images:
            break

    plt.tight_layout()
    plt.show()

plot_from_dataset(train_dataset_static, "Train Dataset without Augmentation", num_images=10)
plot_from_dataset(train_dataset_dynamic, "Train Dataset with Augmentation", num_images=10)

In [ ]:
def compare_datasets(dataset1, dataset2, index):
    # Converti i dataset in liste di batch
    dataset1_batches = list(dataset1)
    dataset2_batches = list(dataset2)

    # Determina l'indice del batch e l'indice all'interno del batch
    batch_size = dataset1_batches[0][0].shape[0]  # Ottieni il batch size dal primo batch
    batch_index = index // batch_size  # Batch contenente l'indice
    within_batch_index = index % batch_size  # Indice all'interno del batch

    # Estrai i batch corrispondenti
    images1, masks1 = dataset1_batches[batch_index]
    images2, masks2 = dataset2_batches[batch_index]

    # Estrai l'immagine e la maschera all'interno del batch
    img1, mask1 = images1[within_batch_index], masks1[within_batch_index]
    img2, mask2 = images2[within_batch_index], masks2[within_batch_index]

    # Converto i tensori in array numpy e rimuovo dimensioni non necessarie
    img1 = img1.numpy().squeeze() if hasattr(img1, "numpy") else img1.squeeze()
    mask1 = mask1.numpy().squeeze() if hasattr(mask1, "numpy") else mask1.squeeze()
    img2 = img2.numpy().squeeze() if hasattr(img2, "numpy") else img2.squeeze()
    mask2 = mask2.numpy().squeeze() if hasattr(mask2, "numpy") else mask2.squeeze()

    # Usa una mappa di colori per le maschere
    cmap = plt.get_cmap("tab20")  # Mappa colori con 20 tonalità diverse
    norm = plt.Normalize(vmin=0, vmax=np.max(mask1))  # Normalizza rispetto al valore massimo della maschera

    # Crea la figura per il confronto
    fig, axes = plt.subplots(4, 1, figsize=(6, 12))  # Layout verticale

    # Dataset1: Immagine
    axes[0].imshow(img1, cmap='gray' if img1.ndim == 2 else None)
    axes[0].set_title(f"Image {index} (Original)", fontsize=10)
    axes[0].axis("off")

    # Dataset1: Maschera colorata
    axes[1].imshow(mask1, cmap=cmap, norm=norm)  # Maschera colorata
    axes[1].set_title(f"Mask {index} (Original)", fontsize=10)
    axes[1].axis("off")

    # Dataset2: Immagine
    axes[2].imshow(img2, cmap='gray' if img2.ndim == 2 else None)
    axes[2].set_title(f"Image {index} (Augmented)", fontsize=10)
    axes[2].axis("off")

    # Dataset2: Maschera colorata
    axes[3].imshow(mask2, cmap=cmap, norm=norm)  # Maschera colorata
    axes[3].set_title(f"Mask {index} (Augmented)", fontsize=10)
    axes[3].axis("off")

    # Mostra la figura
    plt.tight_layout()
    plt.show()

compare_datasets(train_dataset_static, train_dataset_dynamic, 16)

## 🔨 Build the model

In [9]:
class InstanceNormalization(tfkl.Layer):
    def call(self, inputs):
        mean, variance = tf.nn.moments(inputs, axes=[1, 2], keepdims=True)
        return (inputs - mean) / tf.sqrt(variance + 1e-5)

class GroupNormalization(tfkl.Layer):
    def __init__(self, groups=8, epsilon=1e-5, **kwargs):
        super(GroupNormalization, self).__init__(**kwargs)
        self.groups = groups
        self.epsilon = epsilon

    def call(self, inputs):
        # Ottieni la forma dinamica del tensore
        input_shape = tf.shape(inputs)
        batch_size, height, width, channels = input_shape[0], input_shape[1], input_shape[2], input_shape[3]

        # Controlla se i canali sono divisibili per i gruppi
        remainder = tf.math.floormod(channels, self.groups)  # Calcola il resto
        tf.debugging.assert_equal(remainder, 0, message=f"Il numero di canali ({channels}) deve essere divisibile per il numero di gruppi ({self.groups}).")

        # Reshape per raggruppare i canali
        group_shape = [batch_size, height, width, self.groups, channels // self.groups]
        reshaped = tf.reshape(inputs, group_shape)

        # Calcola media e varianza per normalizzazione
        mean, variance = tf.nn.moments(reshaped, axes=[1, 2, 4], keepdims=True)
        normalized = (reshaped - mean) / tf.sqrt(variance + self.epsilon)

        # Ripristina la forma originale
        output = tf.reshape(normalized, input_shape)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

class ResizeLayer(Layer):
    def __init__(self, target_size, **kwargs):
        super(ResizeLayer, self).__init__(**kwargs)
        self.target_size = target_size

    def call(self, inputs):
        return tf.image.resize(inputs, self.target_size)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.target_size[0], self.target_size[1], input_shape[-1])

In [10]:
def unet_block(input_tensor, filters, kernel_size=3, stack=2, name=''):
    x = input_tensor
    for i in range(stack):
        x = tfkl.Conv2D(filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
        x = GroupNormalization(groups=8)(x)
        x = tfkl.Activation(tf.nn.leaky_relu)(x)
    return x

def dense_block(input_tensor, filters, kernel_size=3, growth_rate=32, num_layers=4):
    x = input_tensor
    for i in range(num_layers):
        conv = tfkl.Conv2D(growth_rate, kernel_size, padding='same', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
        conv = tfkl.BatchNormalization()(conv)
        conv = tfkl.Activation(tf.nn.leaky_relu)(conv)
        x = tfkl.Concatenate()([x, conv])
    return x

def se_block(input_tensor, reduction_ratio=16):
    filters = input_tensor.shape[-1]
    se = tfkl.DepthwiseConv2D(kernel_size=3, padding='same', activation='relu')(input_tensor)
    se = tfkl.GlobalAveragePooling2D()(se)
    se = tfkl.Dense(filters // reduction_ratio, activation='relu', kernel_initializer='he_normal')(se)
    se = tfkl.BatchNormalization()(se)
    se = tfkl.Dense(filters, activation='sigmoid', kernel_initializer='he_normal')(se)
    se = tfkl.Reshape((1, 1, filters))(se)
    return tfkl.Multiply()([input_tensor, se])

def refinement_block(x, filters):
    for _ in range(2):  # Due livelli di raffinamento
        x = tfkl.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
        x = tfkl.BatchNormalization()(x)
    return x

# Parallel Dilated Convolutions
def par_dil_conv(input_tensor, filters, kernel_size=3, dilation_rates=(1, 2, 4)):
    branches = []
    for rate in dilation_rates:
        branch = tfkl.Conv2D(filters, kernel_size=kernel_size, dilation_rate=rate, padding='same', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(input_tensor)
        branch = tfkl.BatchNormalization()(branch)
        branch = tfkl.Activation(tf.nn.leaky_relu)(branch)
        branches.append(branch)

    # Concatenazione delle convoluzioni parallele
    output = tfkl.Concatenate()(branches)
    return output

# Atrous Spatial Pyramid Pooling - alternativa a par_dil_conv
def aspp(input_tensor, filters):
    branch1 = tfkl.Conv2D(filters, kernel_size=1, padding='same', activation='relu')(input_tensor)
    branch2 = tfkl.Conv2D(filters, kernel_size=3, dilation_rate=6, padding='same', activation='relu')(input_tensor)
    branch3 = tfkl.Conv2D(filters, kernel_size=3, dilation_rate=12, padding='same', activation='relu')(input_tensor)
    branch4 = tfkl.Conv2D(filters, kernel_size=3, dilation_rate=18, padding='same', activation='relu')(input_tensor)

    # Pooling branch
    pooling = tfkl.GlobalAveragePooling2D()(input_tensor)
    pooling = tfkl.Reshape((1, 1, input_tensor.shape[-1]))(pooling)
    pooling = tfkl.Conv2D(filters, kernel_size=1, padding='same', activation='relu')(pooling)
    pooling = tfkl.UpSampling2D(size=(input_tensor.shape[1], input_tensor.shape[2]))(pooling)

    output = tfkl.Concatenate()([branch1, branch2, branch3, branch4, pooling])
    return tfkl.Conv2D(filters, kernel_size=1, padding='same', activation='relu')(output)

def bottleneck_layer(input_tensor, filters, reduction_ratio=4, dilation_rates=None):
    reduced_filters = filters // reduction_ratio
    bottleneck = tfkl.SeparableConv2D(reduced_filters, kernel_size=3, padding='same', activation='relu')(input_tensor)
    bottleneck = GroupNormalization(groups=8)(bottleneck)
    bottleneck = se_block(bottleneck)
    bottleneck = tfkl.SpatialDropout2D(0.3)(bottleneck)
    return bottleneck

def refinement_block(x, filters):
    x = tfkl.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
    return x

In [11]:
# Dice Loss
def dice_loss(y_true, y_pred, smooth=1e-6):
    # Convert y_true to one-hot encoding
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes, axis=-1)

    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)
    return 1 - dice

# Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    gamma = tf.constant(gamma, dtype=tf.float32)
    alpha = tf.constant(alpha, dtype=tf.float32)

    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        # Converte y_true in one-hot encoding
        y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), depth=y_pred.shape[-1])
        y_true_one_hot = tf.squeeze(y_true_one_hot, axis=-2) if len(y_true_one_hot.shape) > len(y_pred.shape) else y_true_one_hot

        # Calcola la cross-entropy
        cross_entropy = -y_true_one_hot * tf.keras.backend.log(y_pred)

        # Applica il peso focale
        weight = alpha * tf.math.pow((1 - y_pred), gamma)
        loss = weight * cross_entropy

        return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))

    return focal_loss_fixed

# Combined Loss
def combined_loss(y_true, y_pred, dice_weight=0.5, focal_weight=0.5):
    # Calcola Dice Loss
    dice = dice_loss(y_true, y_pred, smooth=1e-6)

    # Calcola Focal Loss
    focal = focal_loss(gamma=2.0, alpha=0.25)(y_true, y_pred)

    # Combina le due loss
    combined = dice_weight * dice + focal_weight * focal

    return combined

## Build DeepLabV3+

In [ ]:
def aspp(inputs, filters=256, rate_scale=1):
    # Controllo forma statica dell'input
    input_shape = K.int_shape(inputs)  # (None, H, W, C)
    H, W = input_shape[1], input_shape[2]

    # Convoluzioni dilatate ASPP
    x1 = tfkl.Conv2D(filters, 1, dilation_rate=1*rate_scale, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(inputs)
    x2 = tfkl.Conv2D(filters, 3, dilation_rate=6*rate_scale, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(inputs)
    x3 = tfkl.Conv2D(filters, 3, dilation_rate=12*rate_scale, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(inputs)
    x4 = tfkl.Conv2D(filters, 3, dilation_rate=18*rate_scale, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(inputs)

    # Global Average Pooling branch
    x5 = tfkl.GlobalAveragePooling2D(keepdims=True)(inputs)  # (None,1,1,C)
    x5 = tfkl.Conv2D(filters, 1, padding='same', activation='relu')(x5)

    # Resize x5 a (H, W)
    x5 = tfkl.Lambda(lambda x: tf.image.resize(x, (H, W), method='bilinear'),
                     output_shape=(H, W, filters))(x5)

    # Concatena tutte le feature map dell'ASPP
    x = tfkl.Concatenate()([x1, x2, x3, x4, x5])
    x = tfkl.Conv2D(filters, 1, padding='same', activation='relu')(x)
    x = GroupNormalization(groups=8)(x)
    x = tfkl.Dropout(0.3)(x)
    return x

def deeplabv3(input_shape, num_classes):
    # Carica ResNet50 come encoder
    backbone = tfk.applications.ResNet50(include_top=False, weights=None, input_shape=input_shape)

    # Ottieni feature per skip connection
    low_level_feature = backbone.get_layer('conv2_block3_out').output
    high_level_feature = backbone.get_layer('conv4_block6_out').output

    # Applica ASPP sul livello alto
    aspp_out = aspp(high_level_feature, filters=256)

    # Ottieni la forma statica del low_level_feature
    low_shape = K.int_shape(low_level_feature)
    LH, LW = low_shape[1], low_shape[2]

    # Ridimensiona aspp_out alla dimensione di low_level_feature
    aspp_out = tfkl.Lambda(lambda x: tf.image.resize(x, (LH, LW), method='bilinear'),
                           output_shape=(LH, LW, 256))(aspp_out)

    # Riduci dimensioni canali di low_level_feature
    low_level_feature = tfkl.Conv2D(48, 1, padding='same', activation='relu')(low_level_feature)
    low_level_feature = tfkl.BatchNormalization()(low_level_feature)

    # Concatenazione
    x = tfkl.Concatenate()([aspp_out, low_level_feature])

    # Decoder
    x = tfkl.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = GroupNormalization(groups=8)(x)
    x = tfkl.Dropout(0.3)(x)
    x = tfkl.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = GroupNormalization(groups=8)(x)
    x = tfkl.Dropout(0.3)(x)

    # Upsample finale alla dimensione di input
    H, W = input_shape[0], input_shape[1]
    x = tfkl.Lambda(lambda img: tf.image.resize(img, (H, W), method='bilinear'),
                    output_shape=(H, W, 256))(x)

    output = tfkl.Conv2D(num_classes, 1, padding='same', activation='softmax')(x)
    model = tfk.Model(backbone.input, output)
    return model

In [15]:
inputs = tfkl.Input(shape=input_shape)

deeplab = deeplabv3(input_shape=input_shape, num_classes=5)

output = deeplab(inputs)

# Modello finale
model = tfk.Model(inputs, output)

# Define the MeanIoU ignoring the background class
mean_iou = tfk.metrics.MeanIoU(num_classes=num_classes, ignore_class=0, sparse_y_pred=False, name='mean_iou')

# Compile the model
optimizer = tfk.optimizers.AdamW(learning_rate=learning_rate, weight_decay=1e-5)

loss = combined_loss

# Compile the model
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[mean_iou]
)

## 🛠️ Train and Save the Model

In [ ]:
# Create an EarlyStopping callback
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_mean_iou',
    patience=patience,
    restore_best_weights=True
)

# Create a LearningRate Scheduler
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5
)

# Store the callbacks in a list
callbacks = [early_stopping, lr_scheduler]

In [ ]:
history = model.fit(
    combined_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    batch_size=batch_size,
    callbacks=callbacks
).history

# Calculate and print the final validation accuracy
final_val_meanIoU = round(max(history['val_mean_iou'])* 100, 2)
print(f'Final validation Mean Intersection Over Union: {final_val_meanIoU}%')

In [27]:
timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
model.save(model_filename)
del model

## 📊 Test the model



In [15]:
model = tfk.models.load_model(model_filename, custom_objects={
        "ResizeLayer": ResizeLayer,
        "InstanceNormalization": InstanceNormalization,
        "GroupNormalization": GroupNormalization,
        'dice_loss': dice_loss,
        'focal_loss': focal_loss,
        'combined_loss': combined_loss,
        'unet_block': unet_block,
        'dense_block': dense_block,
        'par_dil_conv': par_dil_conv,
        'bottleneck_layer': bottleneck_layer,
        'se_block': se_block
    }
)

preds = model.predict(X_test)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 30s 57ms/step
Predictions shape: (10022, 64, 128)


## 💾 Save the predictions

In [16]:
def y_to_df(y) -> pd.DataFrame:
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [18]:
# Create the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)